# Names: Austin He, Maya Lee, Kevin Barcenas Martinez, Luis Gamino


# Pokemon

In this project we will be...

In [1]:
# Standard Headers
# You may add additional headers here if needed
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Enable inline mode for matplotlib so that Jupyter displays graphs
%matplotlib inline

# print your pandas version
pd.__version__ 

'2.2.2'

# Part 1: Data cleaning, data exploration, feature engineering, etc.

In [ ]:
data = pd.read_csv('pokedex.csv', skipinitialspace=True)
print(data.head())

          Image  Index                    Name Type 1  Type 2  Total  HP  \
0  images/1.png      1               Bulbasaur  Grass  Poison    318  45   
1  images/2.png      2                 Ivysaur  Grass  Poison    405  60   
2  images/3.png      3                Venusaur  Grass  Poison    525  80   
3  images/4.png      3  Venusaur Mega Venusaur  Grass  Poison    625  80   
4  images/5.png      4              Charmander   Fire     NaN    309  39   

   Attack  Defense  SP. Atk.  SP. Def  Speed  
0      49       49        65       65     45  
1      62       63        80       80     60  
2      82       83       100      100     80  
3     100      123       122      120     80  
4      52       43        60       50     65  


# Part 2: Modeling

# Part 3: Results and analysis